In [2]:
# llama index to store data to Chroma vector Database local
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core.node_parser import (
    SentenceSplitter
)

# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 64
documents = SimpleDirectoryReader("./data").load_data()

splitter = SentenceSplitter()

#nodes = splitter.get_nodes_from_documents(documents)
#splitter = SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model)


Index data and store to Chroma DB

In [3]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("DB_collection")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context, embed_model=embed_model
#)
VectorStoreIndex.from_documents(documents, storage_context=storage_context, embed_model=embed_model, transformations=[splitter])

llamindex to read chroma db data from chroma db

In [4]:
# llamindex to read chroma db data from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("DB_collection")

vector_store= ChromaVectorStore(chroma_collection=chroma_collection)

vector_store_llamindex = VectorStoreIndex.from_vector_store(
    vector_store,embed_model=embed_model
)
vector_store_llamindex.as_retriever()

In [6]:
vector_store_llamindex.as_query_engine().query("Who is Naresh Dulam?")

Response(response="Aparna is Naresh Dulam's spouse.", source_nodes=[NodeWithScore(node=TextNode(id_='9eed7949-0f15-4937-8dcd-46f1aae083dc', embedding=None, metadata={'page_label': '1', 'file_name': 'Naresh_Dulam.pdf', 'file_path': 'd:\\Data\\Cloud & AI\\CODE-PROJECTS\\llamaindex-llama2\\data\\Naresh_Dulam.pdf', 'file_type': 'application/pdf', 'file_size': 79570, 'creation_date': '2024-04-03', 'last_modified_date': '2024-04-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ce174276-a87c-45f1-99da-0325629b1b67', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Naresh_Dulam.pdf', 'file_path': 'd:\\Data\\Cloud & AI\\CODE-PROJECTS\\llamaindex-llama2\\data\\Naresh_Dulam.pdf', '